# Chapter 43: Pages Router to App Router Migration

The introduction of the App Router in Next.js 13+ represents a fundamental architectural shift from the Pages Router, moving from page-level data fetching to component-level streaming, and from client-side hydration to server-first rendering. While the Pages Router remains fully supported, migrating to the App Router unlocks React Server Components, nested layouts, and improved performance through streaming SSR. This migration requires careful planning as it changes how data flows through your application, how layouts compose, and how client-side interactivity is opt-in rather than default.

By the end of this chapter, you'll master understanding the architectural differences between routing paradigms, planning incremental migration strategies without full rewrites, converting `getServerSideProps` and `getStaticProps` to Server Components, migrating `_app.js` and `_document.js` to Root Layouts, handling loading and error states with suspense boundaries, and maintaining parallel compatibility during transition periods.

## 43.1 Understanding the Architectural Differences

Recognize the fundamental paradigm shifts before beginning migration to avoid common pitfalls.

### Mental Model Comparison

```typescript
// Pages Router: Page-centric, client-hydrated, route-level data
// pages/dashboard.tsx
import { GetServerSideProps } from 'next';

export const getServerSideProps: GetServerSideProps = async ({ req }) => {
  const data = await fetchData(req.cookies.token);
  return { props: { data } }; // Serializes to JSON, hydrates client
};

export default function Dashboard({ data }) {
  // Runs on both server and client
  const [count, setCount] = useState(0);
  return <div>{data.title}</div>;
}

// App Router: Component-centric, server-first, streaming
// app/dashboard/page.tsx
// No getServerSideProps - direct data access in component
async function getData() {
  // Runs only on server (or during static generation)
  const data = await fetchData();
  return data;
}

export default async function DashboardPage() {
  const data = await getData(); // Server Component by default
  
  return (
    <div>
      {data.title}
      <ClientCounter /> {/* Client interactivity isolated */}
    </div>
  );
}

// app/dashboard/client-counter.tsx
'use client'; // Opt-in to client rendering

export function ClientCounter() {
  const [count, setCount] = useState(0);
  return <button onClick={() => setCount(c => c + 1)}>{count}</button>;
}
```

### Key Differences Table

```typescript
// lib/migration/differences.ts

/**
 * Pages Router vs App Router:
 * 
 * 1. Data Fetching:
 *    Pages: getServerSideProps/getStaticProps at page level
 *    App: async components with fetch, cached by default
 * 
 * 2. Rendering:
 *    Pages: Client-side hydration of entire page
 *    App: Server Components stream HTML, Client Components hydrate selectively
 * 
 * 3. State Management:
 *    Pages: useEffect for server state, props drilling
 *    App: Direct await in Server Components, Context in Client Components
 * 
 * 4. Layouts:
 *    Pages: _app.js global layout, manual per-page layouts
 *    App: Nested layouts via folder structure, preserve state across navigations
 * 
 * 5. Error Handling:
 *    Pages: try/catch in data fetching, custom _error.js
 *    App: error.tsx boundaries, not-found.tsx for 404s
 * 
 * 6. Loading States:
 *    Pages: Manual isLoading flags
 *    App: loading.tsx automatic suspense boundaries
 */
```

## 43.2 Incremental Migration Strategy

Adopt the App Router gradually by running both routers simultaneously during the transition.

### Parallel Routing Setup

```typescript
// next.config.js
/** @type {import('next').NextConfig} */
const nextConfig = {
  // Enable App Router alongside Pages Router (default in Next.js 13+)
  experimental: {
    // appDir is stable in Next.js 13.4+, no config needed
  },
  
  // Handle redirects from old pages to new app routes
  async redirects() {
    return [
      // Gradually migrate routes one by one
      // {
      //   source: '/dashboard',
      //   destination: '/dashboard', // Same path, different router
      //   permanent: false,
      // },
    ];
  },
};

// Folder structure for parallel migration:
// app/                    <-- New App Router
//   layout.tsx
//   page.tsx              <-- New homepage
//   dashboard/
//     page.tsx            <-- Migrated dashboard
// pages/                  <-- Legacy Pages Router
//   _app.tsx
//   index.tsx             <-- Old homepage (remove after migration)
//   dashboard.tsx         <-- Remove after app/dashboard is stable
```

### Compatibility Layer

```typescript
// lib/migration/compat.tsx
// Bridge components that work in both routers during transition

'use client';

import { usePathname as useNextPathname } from 'next/navigation';
import { useRouter as useNextRouter } from 'next/navigation';

// Unified router hook that works in both Pages and App Router
export function useCompatRouter() {
  const pathname = useNextPathname();
  const router = useNextRouter();
  
  return {
    pathname,
    push: router.push,
    replace: router.replace,
    refresh: router.refresh,
    back: router.back,
  };
}

// Component that detects which router it's in
export function RouterDetector() {
  const pathname = useNextPathname();
  
  // If pathname exists, we're in App Router
  // If not, we're in Pages Router (use useRouter from next/router instead)
  const isAppRouter = pathname !== null;
  
  return <span>Router: {isAppRouter ? 'App' : 'Pages'}</span>;
}
```

## 43.3 Data Fetching Migration

Convert `getServerSideProps`, `getStaticProps`, and `getServerSidePaths` to Server Components and generateStaticParams.

### From getServerSideProps

```typescript
// Before: Pages Router with getServerSideProps
// pages/user/[id].tsx
import { GetServerSideProps } from 'next';

export const getServerSideProps: GetServerSideProps = async ({ params, req, res }) => {
  const userId = params?.id as string;
  
  // Check auth
  const token = req.cookies.token;
  if (!token) {
    return { redirect: { destination: '/login', permanent: false } };
  }
  
  const user = await db.user.findUnique({ where: { id: userId } });
  
  if (!user) {
    return { notFound: true };
  }
  
  return {
    props: { user }, // Must be serializable JSON
  };
};

export default function UserPage({ user }) {
  return <div>{user.name}</div>;
}

// After: App Router with Server Component
// app/user/[id]/page.tsx
import { notFound, redirect } from 'next/navigation';
import { cookies } from 'next/headers';

// Optional: Generate static params for known users at build time
export async function generateStaticParams() {
  const users = await db.user.findMany({ take: 100 });
  return users.map((user) => ({ id: user.id }));
}

// Dynamic segment configuration
export const dynamic = 'force-dynamic'; // Equivalent to getServerSideProps (no static generation)
// Or: export const revalidate = 60; // Equivalent to getStaticProps with revalidation

async function getUser(id: string) {
  const user = await db.user.findUnique({ where: { id } });
  if (!user) notFound();
  return user;
}

export default async function UserPage({ 
  params 
}: { 
  params: { id: string } 
}) {
  // Auth check directly in component
  const cookieStore = cookies();
  const token = cookieStore.get('token')?.value;
  
  if (!token) {
    redirect('/login');
  }
  
  // Direct data fetching - no props serialization needed
  const user = await getUser(params.id);
  
  // Can pass full objects (including methods) to Client Components if needed
  // But usually better to keep Server Components for data display
  return (
    <div>
      <h1>{user.name}</h1>
      <UserActions userId={user.id} /> {/* Client Component for interactivity */}
    </div>
  );
}

// app/user/[id]/not-found.tsx
export default function NotFound() {
  return <div>User not found</div>;
}
```

### From getStaticProps with Revalidation

```typescript
// Before: Incremental Static Regeneration
// pages/blog/[slug].tsx
export async function getStaticProps({ params }) {
  const post = await getPost(params.slug);
  return {
    props: { post },
    revalidate: 60, // ISR
  };
}

export async function getStaticPaths() {
  return {
    paths: [],
    fallback: 'blocking',
  };
}

// After: App Router with ISR
// app/blog/[slug]/page.tsx
export const revalidate = 60; // ISR equivalent

export async function generateStaticParams() {
  const posts = await getAllPosts();
  return posts.map((post) => ({ slug: post.slug }));
}

export default async function BlogPost({ params }: { params: { slug: string } }) {
  const post = await getPost(params.slug); // Cached, revalidated every 60s
  
  return <article>{post.content}</article>;
}

// On-demand revalidation
// app/api/revalidate/route.ts
import { revalidatePath } from 'next/cache';

export async function POST(request: Request) {
  const { slug } = await request.json();
  revalidatePath(`/blog/${slug}`);
  return Response.json({ revalidated: true });
}
```

## 43.4 Layout Migration

Convert `_app.tsx` and `_document.tsx` to the Root Layout paradigm with nested layout support.

### From _app.js to layout.tsx

```typescript
// Before: Pages Router _app.tsx
// pages/_app.tsx
import { AppProps } from 'next/app';
import { SessionProvider } from 'next-auth/react';
import { QueryClient, QueryClientProvider } from '@tanstack/react-query';
import Layout from '@/components/Layout';

const queryClient = new QueryClient();

export default function MyApp({ Component, pageProps }: AppProps) {
  return (
    <SessionProvider session={pageProps.session}>
      <QueryClientProvider client={queryClient}>
        <Layout>
          <Component {...pageProps} />
        </Layout>
      </QueryClientProvider>
    </SessionProvider>
  );
}

// Before: _document.tsx for HTML customization
// pages/_document.tsx
import { Html, Head, Main, NextScript } from 'next/document';

export default function Document() {
  return (
    <Html lang="en">
      <Head>
        <link rel="manifest" href="/manifest.json" />
      </Head>
      <body>
        <Main />
        <NextScript />
      </body>
    </Html>
  );
}

// After: App Router layout.tsx replaces both
// app/layout.tsx
import { Inter } from 'next/font/google';
import { SessionProvider } from '@/components/session-provider';
import { QueryProvider } from '@/components/query-provider';
import type { Metadata } from 'next';

const inter = Inter({ subsets: ['latin'] });

export const metadata: Metadata = {
  manifest: '/manifest.json',
};

export default function RootLayout({
  children,
}: {
  children: React.ReactNode;
}) {
  return (
    <html lang="en">
      <body className={inter.className}>
        <SessionProvider>
          <QueryProvider>
            {/* Layout can be nested further in subfolders */}
            {children}
          </QueryProvider>
        </SessionProvider>
      </body>
    </html>
  );
}

// After: Nested layout for specific sections
// app/dashboard/layout.tsx
export default function DashboardLayout({
  children,
}: {
  children: React.ReactNode;
}) {
  return (
    <div className="dashboard-container">
      <Sidebar />
      <main>{children}</main>
    </div>
  );
}
```

## 43.5 API Routes Migration

Move API routes from `pages/api` to `app/api` with the new Route Handler pattern.

### Route Handler Conversion

```typescript
// Before: Pages Router API route
// pages/api/users/[id].ts
import type { NextApiRequest, NextApiResponse } from 'next';

export default async function handler(
  req: NextApiRequest,
  res: NextApiResponse
) {
  const { id } = req.query;
  
  if (req.method === 'GET') {
    const user = await db.user.findUnique({ where: { id: id as string } });
    if (!user) {
      return res.status(404).json({ error: 'Not found' });
    }
    return res.json(user);
  }
  
  if (req.method === 'PUT') {
    const updated = await db.user.update({
      where: { id: id as string },
      data: req.body,
    });
    return res.json(updated);
  }
  
  res.setHeader('Allow', ['GET', 'PUT']);
  res.status(405).end(`Method ${req.method} Not Allowed`);
}

// After: App Router Route Handler
// app/api/users/[id]/route.ts
import { NextRequest, NextResponse } from 'next/server';

// GET /api/users/[id]
export async function GET(
  request: NextRequest,
  { params }: { params: { id: string } }
) {
  const user = await db.user.findUnique({ where: { id: params.id } });
  
  if (!user) {
    return NextResponse.json({ error: 'Not found' }, { status: 404 });
  }
  
  return NextResponse.json(user);
}

// PUT /api/users/[id]
export async function PUT(
  request: NextRequest,
  { params }: { params: { id: string } }
) {
  const body = await request.json();
  
  const updated = await db.user.update({
    where: { id: params.id },
    data: body,
  });
  
  return NextResponse.json(updated);
}

// Dynamic route configuration (optional)
export const dynamic = 'force-dynamic'; // or 'auto', 'error'
export const runtime = 'nodejs'; // or 'edge'
```

### Middleware Migration

```typescript
// Before: Pages Router with _middleware.ts or middleware.ts in pages/
// pages/_middleware.ts (legacy location)

// After: Root middleware.ts (works for both routers, but recommend app-specific)
// middleware.ts (project root or app/)
import { NextResponse } from 'next/server';
import type { NextRequest } from 'next/server';

export function middleware(request: NextRequest) {
  // Middleware runs for both Pages and App Router routes
  const token = request.cookies.get('token')?.value;
  
  if (!token && request.nextUrl.pathname.startsWith('/dashboard')) {
    return NextResponse.redirect(new URL('/login', request.url));
  }
  
  return NextResponse.next();
}

export const config = {
  matcher: ['/dashboard/:path*', '/api/protected/:path*'],
};
```

## 43.6 Client Component Migration

Identify which components need interactivity and add the 'use client' directive appropriately.

### Identifying Client Boundaries

```typescript
// Strategy: Start with everything as Server Component, then identify needs

// app/dashboard/page.tsx
import { AnalyticsChart } from './analytics-chart';
import { DataTable } from './data-table';
import { ExportButton } from './export-button';

export default async function DashboardPage() {
  const data = await getData(); // Server-side fetch
  
  return (
    <div>
      <h1>Dashboard</h1>
      
      {/* If AnalyticsChart uses useState/useEffect, it must be Client Component */}
      <AnalyticsChart data={data} />
      
      {/* If DataTable is purely presentational, keep as Server Component */}
      <DataTable data={data} />
      
      {/* ExportButton likely needs onClick, so Client Component */}
      <ExportButton data={data} />
    </div>
  );
}

// app/dashboard/analytics-chart.tsx
'use client'; // Mark as Client Component

import { useState } from 'react';
import { Chart } from 'chart.js';

export function AnalyticsChart({ data }) {
  const [range, setRange] = useState('7d');
  
  // Client-side only code
  useEffect(() => {
    const chart = new Chart(/* ... */);
    return () => chart.destroy();
  }, [data, range]);
  
  return (
    <div>
      <select value={range} onChange={e => setRange(e.target.value)}>
        <option value="7d">7 Days</option>
        <option value="30d">30 Days</option>
      </select>
      <canvas id="chart" />
    </div>
  );
}

// app/dashboard/export-button.tsx
'use client';

export function ExportButton({ data }) {
  const handleExport = () => {
    // Client-side download logic
    const blob = new Blob([JSON.stringify(data)], { type: 'application/json' });
    const url = URL.createObjectURL(blob);
    const a = document.createElement('a');
    a.href = url;
    a.download = 'data.json';
    a.click();
  };
  
  return <button onClick={handleExport}>Export</button>;
}
```

## 43.7 Error and Loading States

Replace manual loading flags and try/catch blocks with declarative loading.tsx and error.tsx.

### Migration to Suspense Boundaries

```typescript
// Before: Manual loading and error states
// pages/dashboard.tsx
export default function Dashboard() {
  const [data, setData] = useState(null);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);
  
  useEffect(() => {
    fetchData()
      .then(setData)
      .catch(setError)
      .finally(() => setLoading(false));
  }, []);
  
  if (loading) return <Spinner />;
  if (error) return <ErrorMessage error={error} />;
  
  return <Content data={data} />;
}

// After: Automatic loading and error handling
// app/dashboard/page.tsx
import { Suspense } from 'react';
import { DataContent } from './data-content';
import { DataSkeleton } from './data-skeleton';

export default function DashboardPage() {
  return (
    <div>
      <h1>Dashboard</h1>
      {/* Static content renders immediately */}
      <DashboardHeader />
      
      {/* Async content wrapped in Suspense */}
      <Suspense fallback={<DataSkeleton />}>
        <DataContent />
      </Suspense>
    </div>
  );
}

// app/dashboard/data-content.tsx
// No 'use client' needed if just fetching data
async function getData() {
  // This fetch is automatically deduped and cached
  const res = await fetch('https://api.example.com/data');
  if (!res.ok) throw new Error('Failed to fetch');
  return res.json();
}

export async function DataContent() {
  const data = await getData(); // Suspends here
  
  return <div>{/* render data */}</div>;
}

// app/dashboard/error.tsx
'use client'; // Error boundaries must be Client Components

import { useEffect } from 'react';

export default function Error({
  error,
  reset,
}: {
  error: Error & { digest?: string };
  reset: () => void;
}) {
  useEffect(() => {
    console.error(error);
  }, [error]);

  return (
    <div>
      <h2>Something went wrong!</h2>
      <button onClick={() => reset()}>Try again</button>
    </div>
  );
}

// app/dashboard/loading.tsx
export default function Loading() {
  return <div>Loading dashboard...</div>;
}
```

## Key Takeaways from Chapter 43

1. **Data Fetching Transformation**: Replace `getServerSideProps` with async Server Components that fetch data directly, and convert `getStaticProps` to `generateStaticParams` with `revalidate` exports. Remove props serialization constraints—Server Components can pass full objects (including non-serializable data) to children.

2. **Layout Architecture**: Migrate `_app.tsx` providers to `app/layout.tsx`, noting that layouts preserve state across navigations (unlike Pages Router remounts). Convert `_document.tsx` customizations to the Root Layout's `<html>` and `<body>` tags with Metadata API for head management.

3. **Client Component Identification**: Start migration by assuming all components are Server Components, then add `'use client'` only when encountering browser APIs (`window`, `document`), React hooks (`useState`, `useEffect`), or event handlers (`onClick`). This minimizes client bundle size.

4. **API Route Evolution**: Replace `NextApiRequest/Response` handlers with exported HTTP method functions (`GET`, `POST`, etc.) in `route.ts` files. Access route parameters through the `params` prop rather than `req.query`, and use `NextRequest/NextResponse` for typed request/response handling.

5. **Error and Loading UI**: Eliminate manual `isLoading` state and try/catch blocks in favor of `loading.tsx` (automatic suspense fallback) and `error.tsx` (error boundary recovery) files placed alongside pages. This creates cleaner component logic and better UX through progressive enhancement.

6. **Parallel Migration**: Run both routers simultaneously by keeping `pages/` for legacy routes while incrementally building `app/` equivalents. Use `dynamic = 'force-dynamic'` for App Router routes that need Pages Router's `getServerSideProps` behavior (no static generation).

7. **Middleware Compatibility**: Existing `middleware.ts` at the project root works for both routers, but be aware that App Router has more granular control via `export const config` in individual route files. Cookie and header handling remains similar but uses the `next/headers` async API in Server Components.

## Coming Up Next

**Chapter 44: Upgrading Next.js Versions**

With your application migrated to the App Router, maintaining currency with Next.js releases becomes crucial. In Chapter 44, we'll explore strategies for upgrading between major versions, using codemods for automated refactoring, handling breaking changes in the App Router evolution, dependency management strategies, testing procedures during upgrades, and rollback strategies when issues arise. You'll learn how to stay current with the framework's rapid evolution while minimizing disruption to production applications.